<a href="https://colab.research.google.com/github/NoVeR-18/DSL/blob/main/Job2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def productiveSymbols(grammar) -> set:
    productive = set()
    prev_count = None
    def unproductiveSymbols(rule: str) -> bool:
        return all(map(lambda s: s in productive or s in grammar['toks'], rule))
    while len(productive) != prev_count:
        prev_count = len(productive)
        productive_symbols = [var for var, definition in grammar['vars'].items() if
                              list(filter(unproductiveSymbols, definition)) != []]
        productive = productive.union(set(productive_symbols))

    return productive


def reachableSymbols(grammar) -> set:
    reachable = {grammar['hvar']}
    prev_count = None
    while len(reachable) != prev_count:
        prev_count = len(reachable)
        for var, definition in grammar['vars'].items():
            if var in reachable:
                for rule in definition:
                    reachable = reachable.union(set(filter(lambda s: s not in grammar['toks'], rule)))
    return reachable
from copy import deepcopy


"""функції для прибирання сторонніх нетерміналів"""

def removeSymbols(grammar):
    grammar_copy = deepcopy(grammar)
    productive = productiveSymbols(grammar)
    grammar_copy['vars'] = {
        var: [rule for rule in definition if all(map(lambda s: s in productive or s in grammar['toks'], rule))]
        for var, definition in grammar_copy['vars'].items()}

    reachable = reachableSymbols(grammar_copy)
    grammar_copy['vars'] = {
        var: [rule for rule in definition if all(map(lambda s: s in reachable or s in grammar['toks'], rule))]
        for var, definition in grammar_copy['vars'].items() if var in reachable}
    return grammar_copy


"""функції для визначення зникаючих символів """


def nullableSymbols(grammar) -> set:
    nullable = set()
    prev_count = None
    def allNullableSymbols(rule: str) -> bool:
        return all(map(lambda s: s in nullable or s == (0, ''), rule))
    while len(nullable) != prev_count:
        prev_count = len(nullable)
        nullable_symbols = [var for var, definition in grammar['vars'].items()
                            if list(filter(allNullableSymbols, definition)) != []]
        nullable = nullable.union(set(nullable_symbols))
    return nullable


grammar = {
    'toks': {(0, ''), (1, 'a'), (2, 'b'), (3, 'c')},
    'vars': {
        'S': [[(1, 'a'), 'M'],
              [(1, 'a'), 'S'],
              ['P']],
        'K': [[(3, 'c')],
              [(0, '')]],
        'M': [['M', 'P']],
        
        'P': [[(3, 'c')]]
    },
    'hvar': 'S'
}
print(removeSymbols(grammar))
print('Nullable symbols ' + str(nullableSymbols(grammar)))
